In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt-get install build-essential swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [3]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   207  100   207    0     0   1101      0 --:--:-- --:--:-- --:--:--  1095


In [4]:
!pip3 install auto-sklearn

In [5]:
import sklearn.datasets
import sklearn.metrics
 
import autosklearn.classification
 
import numpy as np
import pandas as pd

/usr/local/lib/python3.7/dist-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


# Data Loading

In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate-414/train.csv', header = 0)
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate-414/test.csv', header = 0)

In [7]:
y = train_df["Outcome"].values
X = train_df.drop(["index","Outcome"], axis=1).values

In [8]:
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=1)

# Build and fit a regressor

In [9]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

In [10]:
#automl = autosklearn.classification.AutoSklearnClassifier(
automl = AutoSklearn2Classifier(
    time_left_for_this_task=7200, # long
    #time_left_for_this_task=120, # short
    #per_run_time_limit=60,
    #resampling_strategy="cv",
    #resampling_strategy_arguments={"folds": 5},
    metric=autosklearn.metrics.balanced_accuracy,
    #metric=autosklearn.metrics.accuracy,
    scoring_functions=[autosklearn.metrics.balanced_accuracy,
                       autosklearn.metrics.accuracy,
                       autosklearn.metrics.f1,
                       autosklearn.metrics.roc_auc,
                       autosklearn.metrics.precision,
                       autosklearn.metrics.recall],
)
automl.fit(X_train, y_train, X_test, y_test, feat_type=["Numerical" for i in range(X_train.shape[1])])

Using no backup selector


AutoSklearn2Classifier(metric=balanced_accuracy,
                       scoring_functions=[balanced_accuracy, accuracy, f1,
                                          roc_auc, precision, recall],
                       time_left_for_this_task=7200)

In [11]:
def get_metric_result(cv_results):
    results = pd.DataFrame.from_dict(cv_results)
    results = results[results['status'] == "Success"]
    cols = ['rank_test_scores', 'param_classifier:__choice__', 'mean_test_score']
    cols.extend([key for key in cv_results.keys() if key.startswith('metric_')])
    return results[cols]
print(get_metric_result(automl.cv_results_).to_string(index=False))

 rank_test_scores param_classifier:__choice__  mean_test_score  metric_balanced_accuracy  metric_accuracy  metric_f1  metric_roc_auc  metric_precision  metric_recall
              209                         sgd         0.532509                  0.532509         0.496874   0.354626        0.532509          0.252681       0.599322
              192                         sgd         0.582710                  0.582710         0.707917   0.357893        0.583266          0.369479       0.347582
              207                         sgd         0.534949                  0.534949         0.591684   0.328776        0.548166          0.266680       0.428570
              173                         sgd         0.612084                  0.612084         0.571863   0.428381        0.664435          0.311121       0.687497
              132                         sgd         0.665287                  0.665287         0.647932   0.482699        0.722118          0.370136       0.697828
    

# Print the final ensemble constructed by auto-sklearn

In [12]:
print(automl.show_models())

[(0.400000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:random_forest:bootstrap': 'False', 'classifier:random_forest:criterion': 'entropy', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.1036663012470118, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decrease': 0.0, 'classifier:random_forest:min_samples_leaf': 11, 'classifier:random_forest:min_samples_split': 5, 'classifier:random_forest:min_weight_fraction_leaf': 0.0, 

# Report statistics about the search

In [13]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 2f23e8b4-7b91-11eb-82a0-0242ac1c0002
  Metric: balanced_accuracy
  Best validation score: 0.726919
  Number of target algorithm runs: 251
  Number of successful target algorithm runs: 251
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0



# Get the Score of the final ensemble

In [14]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.7616666666666667


# Perform a refit

In [15]:
REFIT = False


In [16]:
if REFIT:
  automl.refit(X_train.copy(), y_train.copy())
  predictions = automl.predict(X_test)
  print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

# Submit

In [17]:
preds = pd.DataFrame(automl.predict(test_df.drop("index", axis=1).values))
submit = pd.concat([test_df["index"], preds[0]], axis=1)
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/signate-414/submit_autosklearn2-2-balanced-norefit-7200.csv', header=False, index=False)

In [18]:
pd.DataFrame(preds)

,0
0,1
1,0
2,0
3,0
4,0
...,...
1995,0
1996,0
1997,1
1998,0
